## Import Libraries

In [1]:
import matplotlib.pyplot as plt
from numpy import sin, cos, sqrt, pi, linspace, arange, deg2rad, rad2deg, array, arcsin, arccos,sort, argsort, argwhere, argmin, argmax, interp, concatenate, zeros
from scipy.spatial import distance
from numpy import linalg as LA
import warnings
warnings.simplefilter('ignore')

def sec(x):
    return 1/cos(x)
def tan(x):
    return sin(x)/cos(x)

## Initialize Parameters 

In [19]:
n      = 1.33
c      = 299792458/n
h      = 4.5
D      = 7.3   #--------------------- diameter of tank
R      = 7.3/2 #--------------------- radius of tank
v      = n * c
times  = linspace(1e-12,1e-7,200000)

D1     = array([0, 0, 0])  #------------------------------------ Central    PMT number 1 
D2     = array([1.85*cos(2*pi/3) , 1.85*sin(2*pi/3), 0]) #------ Non-Radial PMT number 2
D3     = array([1.85*cos(4*pi/3) , 1.85*sin(4*pi/3), 0]) #------ Non-Radial PMT number 3
D4     = array([1.85*cos(0     ) , 1.85*sin(0     ), 0]) #------ Radial     PMT number 4
x1,y1,z1 = [1.2,1.2,h] #----------------------------------------- Coordinates of particle at top tanker lid      #------PARAM
theta  = 145   #----------------------------------------- Zenith  varies from 90 to 180                    #------PARAM
phi    =  10   #----------------------------------------- Azimuth varies from  0 to 360                    #------PARAM
theta  = deg2rad(theta) #-------------------------------- Convert zenith to radians
phi    = deg2rad(phi  ) #-------------------------------- Convert azimuth to radians

x2     = x1 + h * tan(theta) * cos(phi) # ------------------- Calculate bottom x coordinate
y2     = y1 + h * tan(theta) * sin(phi) #-------------------- Calculate bottom y coordinate

A      = array([x1,y1,z1])
B      = array([x2,y2,0 ])

nBD1   = LA.norm(B-D1)
nBD2   = LA.norm(B-D2)
nBD3   = LA.norm(B-D3)
nBD4   = LA.norm(B-D4)

theta2    = pi - theta
del theta
theta     = theta2

print('A   : ', A)
print('B   : ', B)
print('D1  : ', D1)
print('D2  : ', D2)
print('D3  : ', D3)
print('D4  : ', D4)
print('BD1 : %.2f m\nBD2 : %.2f m\nBD3 : %.2f m\nBD4 : %.2f m'%(nBD1,nBD2,nBD3,nBD4) )

A   :  [1.2 1.2 4.5]
B   :  [-1.90306416  0.65284607  0.        ]
D1  :  [0 0 0]
D2  :  [-0.925     1.602147  0.      ]
D3  :  [-0.925    -1.602147  0.      ]
D4  :  [1.85 0.   0.  ]
BD1 : 2.01 m
BD2 : 1.36 m
BD3 : 2.46 m
BD4 : 3.81 m


## Computations 

In [ ]:
def entry_brightness(h,c,v,theta,L):
    z        = h/cos(theta)
    k        = (sqrt( L**2 + z**2 - 2*z*L*cos(pi/2-theta)))
    t_shower = 0
    t_light  = k/c
    t_total  = t_shower + t_light
    tt       = t_total
    den      = (c**2-v**2)
    num1_z   = -c**2*tt*v + c**2*h*sec(theta) - L*v**2*sin(theta)
    num2_z   = sqrt(v**2* ( c*c*L*L - L*L*v*v + c*c*tt*tt*v*v - 2*c*c*h*tt*v*sec(theta)  \
               + c*c*h*h*sec(theta)**2 + 2*c*c*L*tt*v*sin(theta) + L*L*v*v*sin(theta)**2 \
               - 2*c*c*h*L*tan(theta)))
    zp       = ((num1_z - num2_z) / den)
    v_A      = v*v*(tt*v-h*sec(theta)+L*sin(theta))
    v_B      = num2_z
    vp       = (c*c*v) * (1+(v_A/v_B)) / (-den)
    kp       = sqrt( L**2 + zp**2 - 2*zp*L*cos(pi/2-theta))
    alphap   = (arccos( (zp*zp + kp*kp - L*L) / (2*zp*kp) ))
    vtp      = vp * sin(alphap)
    omegap   = (vtp / kp)
    bp       = abs(omegap / (kp**2))
    return bp

#==========================================================================================================================================
#==========================================================================================================================================

def plus_b_vs_phi(phip,bp,color,mylabel,phi):
    plt.plot(phip, bp, c=color, ls='-', label=mylabel)
    plt.axhline(1,  c='k', ls=':')
    plt.axvline(phi,c='k', ls=':')
    plt.text(phi+1, 0.5, r'$\theta_C$ = {}'.format(round(phi,2))     , fontsize=18)
    plt.xlabel(r'angular locations $\phi_{C}\;(in\;degrees)$', fontsize=18)
    plt.ylabel(r'brightness ($b/b_{entry}$) '       , fontsize=18).set_rotation(90)
    plt.tick_params(axis='both', direction='in', labelsize=18)        
    plt.yscale('log')
def minus_b_vs_phi(phim,bm,color,mylabel,phi):
    plt.plot(phim, bm, c=color, ls='--',label=mylabel)
    plt.axhline(1,  c='k', ls=':')
    plt.axvline(phi,c='k', ls=':')
    plt.text(phi+1, 0.5, r'$\theta_C$ = {}'.format(round(phi,2))     , fontsize=18)
    plt.xlabel(r'angular locations $\phi_{C}\;(in\;degrees)$', fontsize=18)
    plt.ylabel(r'brightness ($b/b_{entry}$) '       , fontsize=18).set_rotation(90)
    plt.tick_params(axis='both', direction='in', labelsize=18)        
    plt.yscale('log')
def both_b_vs_phi(phip,bp,phim,bm,color,mylabel,phi):
    plt.plot(phip, bp, c=color, ls='-', label=mylabel)
    plt.plot(phim, bm, c=color, ls='--')
    plt.axhline(1,  c='k', ls=':')
    plt.axvline(phi,c='k', ls=':')
    plt.text(phi+1, 0.5, r'$\theta_C$ = {}'.format(round(phi,2))     , fontsize=18)
    plt.xlabel(r'angular locations $\phi_{C}\;(in\;degrees)$', fontsize=18)
    plt.ylabel(r'brightness ($b/b_{entry}$) '       , fontsize=18).set_rotation(90)
    plt.tick_params(axis='both', direction='in', labelsize=18)        
    plt.yscale('log')
    
#==========================================================================================================================================
#==========================================================================================================================================
    
def plus_b_vs_z(zp,bp,color,mylabel,zc):  # Use times  = linspace(1e-12,1e-7,2000000)
    plt.plot(zp, bp, c=color, ls='-', label=mylabel, lw=2.5)
    if (0<=zc<=h):
        plt.text(zc+0.03, 5000, r'$z_C$ = {}'.format(round(zc,2))     , fontsize=18)
        plt.axvline(zc_act,c='k',ls=':')
    plt.xlabel(r'image heights $z_{pm}\;(in\;meters)$', fontsize=18)
    plt.ylabel(r'brightness ($b/b_{entry}$) '       , fontsize=18).set_rotation(90)
    plt.tick_params(axis='both', direction='in', labelsize=18)        
    plt.yscale('log')
    plt.xlim(0,h)
def minus_b_vs_z(zm,bm,color,mylabel,zc):  # Use times  = linspace(1e-12,1e-7,2000000)
    plt.plot(zm, bm, c=color, ls='--',label=mylabel, lw=2.5)
    if (0<=zc<=h):
        plt.text(zc+0.03, 5000, r'$z_C$ = {}'.format(round(zc,2))     , fontsize=18)
        plt.axvline(zc_act,c='k',ls=':')
    plt.xlabel(r'image heights $z_{pm}\;(in\;meters)$', fontsize=18)
    plt.ylabel(r'brightness ($b/b_{entry}$) '       , fontsize=18).set_rotation(90)
    plt.tick_params(axis='both', direction='in', labelsize=18)        
    plt.yscale('log')
    plt.xlim(0,h)
def both_b_vs_z(zp,bp,zm,bm,color,mylabel,zc):  # Use times  = linspace(1e-12,1e-7,2000000)
    plt.plot(zp, bp, c=color, ls='-', label=mylabel, lw=2.5)
    plt.plot(zm, bm, c=color, ls='--', lw=2.5)
    if (0<=zc<=h):
        plt.text(zc+0.03, 5000, r'$z_C$ = {}'.format(round(zc,2))     , fontsize=18)
        plt.axvline(zc_act,c='k',ls=':')
    plt.xlabel(r'image heights $z_{pm}\;(in\;meters)$', fontsize=18)
    plt.ylabel(r'brightness ($b/b_{entry}$) '       , fontsize=18).set_rotation(90)
    plt.tick_params(axis='both', direction='in', labelsize=18)        
    plt.yscale('log')
    plt.xlim(0,h)
    
#==========================================================================================================================================
#==========================================================================================================================================

def plus_b_vs_t(t,bp,color,mylabel):  # Use times  = linspace(1e-12,1e-7,2000000)
    plt.plot(t, bp, c=color, ls='-', label=mylabel, lw=2.5)
    plt.axhline(1, c='k', ls=':')
    plt.axvline(0, c='k', ls=':')
    plt.xlabel(r'time (ns)', fontsize=18)
    plt.ylabel(r'brightness ($b/b_{entry}$) '       , fontsize=18).set_rotation(90)
    plt.tick_params(axis='both', direction='in', labelsize=18)        
    plt.yscale('log')
    #plt.ylim(0,1e3)
def minus_b_vs_t(t,bm,color,mylabel):  # Use times  = linspace(1e-12,1e-7,2000000)
    plt.plot(t, bm, c=color, ls='--',label=mylabel, lw=2.5)
    plt.axhline(1, c='k', ls=':')
    plt.axvline(0, c='k', ls=':')
    plt.xlabel(r'time (ns)', fontsize=18)
    plt.ylabel(r'brightness ($b/b_{entry}$) '       , fontsize=18).set_rotation(90)
    plt.tick_params(axis='both', direction='in', labelsize=18)        
    plt.yscale('log')
    #plt.ylim(0,1e3)
def both_b_vs_t(tp,bp,tm,bm,color,mylabel):  # Use times  = linspace(1e-12,1e-7,2000000)
    plt.plot(tp, bp, c=color, ls='-', label=mylabel, lw=2.5)
    plt.plot(tm, bm, c=color, ls='--', lw=2.5)
    plt.axhline(1, c='k', ls=':')
    plt.axvline(0, c='k', ls=':')
    plt.xlabel(r'time (ns)', fontsize=18)
    plt.ylabel(r'brightness ($b/b_{entry}$) '       , fontsize=18).set_rotation(90)
    plt.tick_params(axis='both', direction='in', labelsize=18)        
    plt.yscale('log')
    #plt.ylim(0,1e3)
    
#==========================================================================================================================================
#==========================================================================================================================================
    
def plus_phi_vs_t(tp,phip,color,mylabel,phi):
    plt.plot(tp, phip, c=color, ls='-' , label=mylabel, lw=2)
    plt.xlabel(r'time (ns)', fontsize=18)
    plt.axhline(phi, c='k', ls=':')
    plt.axvline(0, c='k', ls=':')
    plt.text(20.5, phi+1.5, r'$\theta_C = %.2f ^{\circ}$'%phi , fontsize=15)
    plt.ylabel(r'angular locations $\phi_{pm}\;(in\;degrees)$'     , fontsize=18).set_rotation(90)
    plt.tick_params(axis='both', direction='in', labelsize=18)
    plt.ylim(0,80)
    
def minus_phi_vs_t(tm,phim,color,mylabel,phi):
    plt.plot(tm, phim, c=color, ls='-.' , label=mylabel,lw=2)
    plt.xlabel(r'time (ns)', fontsize=18)
    plt.axhline(phi, c='k', ls=':')
    plt.axvline(0, c='k', ls=':')
    plt.text(20.5, phi+1.5, r'$\theta_C = %.2f ^{\circ}$'%phi , fontsize=15)
    plt.ylabel(r'angular locations $\phi_{pm}\;(in\;degrees)$'     , fontsize=18).set_rotation(90)
    plt.tick_params(axis='both', direction='in', labelsize=18)
    plt.ylim(0,80)
    
def both_phi_vs_t(tp,phip,tm,phim,color,mylabel,phi):
    plt.plot(tp, phip, c=color, ls='-' , label=mylabel,lw=2)
    plt.plot(tm, phim, c=color, ls='-.',lw=2)
    plt.xlabel(r'time (ns)', fontsize=18)
    plt.axhline(phi, c='k', ls=':')
    plt.axvline(0, c='k', ls=':')
    plt.text(20.5, phi+1.5, r'$\theta_C = "%.2f" ^{\circ}$'%phi , fontsize=15)
    plt.ylabel(r'angular locations $\phi_{pm}\;(in\;degrees)$'     , fontsize=18).set_rotation(90)
    plt.tick_params(axis='both', direction='in', labelsize=18)
    plt.ylim(0,80)
    plt.xticks(arange(18,25.2,1))
    
#==========================================================================================================================================
#==========================================================================================================================================
def plus_z_vs_t(tp,zp,color,mylabel,zc):
    plt.plot(tp, zp, c=color, ls='-' , label=mylabel, lw=2.5)
    plt.xlabel(r'time (ns)', fontsize=18)
    if (0<=zc<=h):
        plt.text(30, zc+0.05, r'$z_C = %.2f $'%(round(zc,2)) , fontsize=15)
        plt.axhline(zc, c='k', ls=':')
    plt.axvline(0, c='k', ls=':')
    plt.ylabel(r'image heights $z_{pm} (in\;meters)$'     , fontsize=18).set_rotation(90)
    plt.tick_params(axis='both', direction='in', labelsize=18)
    plt.ylim(0,h)
def minus_z_vs_t(tm,zm,color,mylabel,zc):
    plt.plot(tm, zm, c=color, ls='-.' , label=mylabel, lw=2.5)
    plt.xlabel(r'time (ns)', fontsize=18)
    if (0<=zc<=h):
        plt.text(30, zc+0.05, r'$z_C = %.2f $'%(round(zc,2)) , fontsize=15)
        plt.axhline(zc, c='k', ls=':')
    plt.axvline(0, c='k', ls=':')
    plt.ylabel(r'image heights $z_{pm} (in\;meters)$'     , fontsize=18).set_rotation(90)
    plt.tick_params(axis='both', direction='in', labelsize=18)
    plt.ylim(0,h)
def both_z_vs_t(tp,zp,tm,zm,color,mylabel,zc):
    plt.plot(tp, zp, c=color, ls='-' , label=mylabel, lw=2.5)
    plt.plot(tm, zm, c=color, ls='-.', lw=2.5)
    plt.xlabel(r'time (ns)', fontsize=18)
    if (0<=zc<=h):
        plt.text(30, zc+0.05, r'$z_C = %.2f $'%(round(zc,2)) , fontsize=15)
        plt.axhline(zc, c='k', ls=':')
    plt.axvline(0, c='k', ls=':')
    plt.ylabel(r'image heights $z_{pm} (in\;meters)$'     , fontsize=18).set_rotation(90)
    plt.tick_params(axis='both', direction='in', labelsize=18)
    plt.ylim(0,h)
    plt.yticks(arange(0,7.1,0.5))

In [ ]:
def entry_brightness(h,c,v,theta,L):
    z        = h/cos(theta)
    k        = (sqrt( L**2 + z**2 - 2*z*L*cos(pi/2-theta)))
    t_shower = 0
    t_light  = k/c
    t_total  = t_shower + t_light
    tt       = t_total
    den      = (c**2-v**2)
    num1_z   = -c**2*tt*v + c**2*h*sec(theta) - L*v**2*sin(theta)
    num2_z   = sqrt(v**2* ( c*c*L*L - L*L*v*v + c*c*tt*tt*v*v - 2*c*c*h*tt*v*sec(theta)  \
               + c*c*h*h*sec(theta)**2 + 2*c*c*L*tt*v*sin(theta) + L*L*v*v*sin(theta)**2 \
               - 2*c*c*h*L*tan(theta)))
    zp       = ((num1_z - num2_z) / den)
    v_A      = v*v*(tt*v-h*sec(theta)+L*sin(theta))
    v_B      = num2_z
    vp       = (c*c*v) * (1+(v_A/v_B)) / (-den)
    kp       = sqrt( L**2 + zp**2 - 2*zp*L*cos(pi/2-theta))
    alphap   = (arccos( (zp*zp + kp*kp - L*L) / (2*zp*kp) ))
    vtp      = vp * sin(alphap)
    omegap   = (vtp / kp)
    bp       = abs(omegap / (kp**2))
    return bp

def plus_phi_vs_t(tp,phip,color,mylabel,phi):
    plt.plot(tp, phip, c=color, ls='-' , label=mylabel, lw=2)
    plt.xlabel(r'time (ns)', fontsize=18)
    plt.axhline(phi, c='k', ls=':')
    plt.axvline(0, c='k', ls=':')
    #plt.text(20.5, phi+1.5, r'$\theta_C = %.2f ^{\circ}$'%phi , fontsize=15)
    plt.ylabel(r'angular locations $\phi_{pm}\;(in\;degrees)$'     , fontsize=18).set_rotation(90)
    plt.tick_params(axis='both', direction='in', labelsize=18)
    #plt.ylim(0,80)
    
def minus_phi_vs_t(tm,phim,color,mylabel,phi):
    plt.plot(tm, phim, c=color, ls='-.' , label=mylabel,lw=2)
    plt.xlabel(r'time (ns)', fontsize=18)
    plt.axhline(phi, c='k', ls=':')
    plt.axvline(0, c='k', ls=':')
    #plt.text(20.5, phi+1.5, r'$\theta_C = %.2f ^{\circ}$'%phi , fontsize=15)
    plt.ylabel(r'angular locations $\phi_{pm}\;(in\;degrees)$'     , fontsize=18).set_rotation(90)
    plt.tick_params(axis='both', direction='in', labelsize=18)
    #plt.ylim(0,80)
    
def both_phi_vs_t(tp,phip,tm,phim,color,mylabel,phi):
    plt.plot(tp, phip, c=color, ls='-' , label=mylabel,lw=2)
    plt.plot(tm, phim, c=color, ls='-.',lw=2)
    plt.xlabel(r'time (ns)', fontsize=18)
    plt.axhline(phi, c='k', ls=':')
    plt.axvline(0, c='k', ls=':')
    #plt.text(20.5, phi+1.5, r'$\theta_C = "%.2f" ^{\circ}$'%phi , fontsize=15)
    plt.ylabel(r'angular locations $\phi_{pm}\;(in\;degrees)$'     , fontsize=18).set_rotation(90)
    plt.tick_params(axis='both', direction='in', labelsize=18)
    #plt.ylim(0,80)
    #plt.xticks(arange(18,25.2,1))

In [ ]:
#%matplotlib
plt.figure(figsize=(15,12))
#plotme    = 'b vs z'
#plotme    = 'b vs phi'
#plotme    = 'b vs t'
#plotme    = 'b vs z'
#plotme    = 'z vs t'
plotme    = 'phi vs t'

detectors            = [1]#,2,3,4]
detector_distances   = [nBD1]#,nBD2,nBD3,nBD4]
colors               = ['k']#,'r','b','g']
labels               = ['Central Detector']#, "Detector 2",'Detector 3','Detector 4']

for detector, BD, color, mylabel in zip(detectors,  detector_distances, [d1], ['k'], [1], ['central'], [c1]):
    print(detector,"\n\n")
    L      = BD
    num_zc = -sqrt(-c**4 * L**2 * cos(theta)**2 + c**2 * L**2 * v**2 * cos(theta)**2) + c**2*L*sin(theta) - L*v**2*sin(theta)
    den    = (c**2-v**2)
    zc     = (num_zc/den)
    zc_act = zc * cos(theta)
    print('zc : {} m'.format(round(zc_act,3)))
    
    T, ZP, ZM, BP, BM, PHIP, PHIM = [], [], [], [], [], [], []
    for iii,t in enumerate(times):
        z        = h/cos(theta) - v*t
        k        = (sqrt( L**2 + z**2 - 2*z*L*cos(pi/2-theta)))
        t_shower = t
        t_light  = k/c
        t_total  = t_shower + t_light
        tt       = t_total

        num1_z   = -c**2*tt*v + c**2*h*sec(theta) - L*v**2*sin(theta)
        num2_z   = sqrt(v**2* ( c*c*L*L - L*L*v*v + c*c*tt*tt*v*v - 2*c*c*h*tt*v*sec(theta) + c*c*h*h*sec(theta)**2 + 2*c*c*L*tt*v*sin(theta) + L*L*v*v*sin(theta)**2 - 2*c*c*h*L*tan(theta)))
        zp       = ((num1_z - num2_z) / den)
        zm       = ((num1_z + num2_z) / den)
        v_A      = v*v*(tt*v-h*sec(theta)+L*sin(theta))
        v_B      = num2_z
        vp       = (c*c*v) * (1+(v_A/v_B)) / (-den)
        vm       = (c*c*v) * (-1+(v_A/v_B)) / den        
        kp       = sqrt( L**2 + zp**2 - 2*zp*L*cos(pi/2-theta))
        km       = sqrt( L**2 + zm**2 - 2*zm*L*cos(pi/2-theta))
        
        zx       = v*t
        ratio    = zx/nAB
        A        = A
        B        = B
        X        = array([ (1-ratio)*A[0] + ratio*B[0] , (1-ratio)*A[1] + ratio*B[1] , (1-ratio)*A[2]+ratio*B[2]])
        D        = array(c1)
        
        ratio2   = (nAB-zp)
        ratio3   = (nAB-zm)
        XP       = array([ (1-ratio2)*A[0] + ratio2*B[0] , (1-ratio2)*A[1] + ratio2*B[1] , (1-ratio2)*A[2]+ratio2*B[2]])
        XM       = array([ (1-ratio3)*A[0] + ratio3*B[0] , (1-ratio3)*A[1] + ratio3*B[1] , (1-ratio3)*A[2]+ratio3*B[2]])
        
        alphap   = arccos( sum((B-XP)*(D-XP)) / (LA.norm(B-XP)*LA.norm(D-XP)) )
        alpham   = arccos( sum((B-XM)*(D-XM)) / (LA.norm(B-XM)*LA.norm(D-XM)) )
        
        phip     = arccos( sum((B-D)*(D-XP)) / (LA.norm(B-D)*LA.norm(D-XP)) )
        phim     = arccos( sum((B-D)*(D-XM)) / (LA.norm(B-D)*LA.norm(D-XM)) )
        
        if (phip>pi/2):
            phip=phip-pi/2
        if (phim>pi/2):
            phim=phim-pi/2
        #alphap   = (arccos( (zp*zp + kp*kp - L*L) / (2*zp*kp) ))
        #alpham   = (arccos( (zm*zm + km*km - L*L) / (2*zm*km) ))
        
        #phip     = 90+theta-rad2deg(alphap)
        #phim     = 90+theta-rad2deg(alpham)               
        vtp      = vp * sin(alphap)
        vtm      = vm * sin(alpham)
        omegap   = (vtp / kp)
        omegam   = (vtm / km)
        bp       = abs(omegap / (kp**2))
        bm       = abs(omegam / (km**2))
        
        ZM.append(zm*cos(theta))
        ZP.append(zp*cos(theta))
        BM.append(bm)            
        BP.append(bp)
        PHIP.append(rad2deg(phip))
        PHIM.append(rad2deg(phim))
        T.append(tt)
        
        if (iii%10000==0):
            print(round(tt*1e9,3),round(zp,3),round(zm,3),round(rad2deg(alphap),3),round(rad2deg(alpham),3),round(rad2deg(phip),3),round(rad2deg(phim),3),sep="  ")
        
    K = sqrt( L**2 + zc**2 - 2*zc*L*cos(pi/2-theta))
    ALPHA   = (arccos( (zc*zc + K*K - L*L) / (2*zc*K) ))
    PHI     = 90-theta-rad2deg(ALPHA)

    ZP, ZM, TT, BP, BM, PHIP, PHIM = array(ZP), array(ZM), array(T)*1e9, array(BP), array(BM), array(PHIP), array(PHIM)
    
    conp = [(ZP>=0) & (ZP<=h)]
    conm = [(ZM>=0) & (ZM<=h)]
    pluslen  = len (ZP[conp])
    minuslen = len (ZM[conm])
    
    print(pluslen,minuslen)
    if detector==1:
        den_bright = entry_brightness(h=h,c=c,v=v,theta=theta,L=L)
    
    if pluslen == 0:
        if minuslen == 0:
            print("Images outside tank. Skipping...")
        else:
            print('Only image moving down. Processing...')
            TTm,ZM,BM,PHIM = TT[conm],ZM[conm],BM[conm]/den_bright,PHIM[conm]
            if plotme=='b vs phi':
                minus_b_vs_phi(phim=PHIM,bm=BM,color=color,mylabel=mylabel,phi=PHI)
            elif plotme=='b vs z':
                minus_b_vs_z(zm=ZM,bm=BM,color=color,mylabel=mylabel,zc=zc_act)
            elif plotme=='b vs t':
                minus_b_vs_t(t=TT,bm=BM,color=color,mylabel=mylabel)
            elif plotme=='phi vs t':
                minus_phi_vs_t(tm=TTm,phim=PHIM,color=color,mylabel=mylabel,phi=PHI)
            elif plotme=='z vs t':
                minus_z_vs_t(tm=TT,zm=ZM,color=color,mylabel=mylabel,zc=zc_act)
    elif pluslen != 0:
        if minuslen == 0:
            print('Only image moving up. Processing...')
            TTp,ZP,BP,PHIP = TT[conp],ZP[conp],BP[conp]/den_bright,PHIP[conp]
            if plotme=='b vs z':
                plus_b_vs_phi(phip=PHIP,bp=BP,color=color,mylabel=mylabel,phi=PHI)
            elif plotme=='b vs z':
                plus_b_vs_z(zp=ZP,bp=BP,color=color,mylabel=mylabel,zc=zc_act)
            elif plotme=='b vs t':
                plus_b_vs_t(t=TT,bp=BP,color=color,mylabel=mylabel)
            elif plotme=='phi vs t':
                plus_phi_vs_t(tp=TTp,phip=PHIP,color=color,mylabel=mylabel,phi=PHI)
            elif plotme=='z vs t':
                plus_z_vs_t(tp=TT,zp=ZP,color=color,mylabel=mylabel,zc=zc_act)
        else:
            print('Both images moving. Processing...')
            TTm,ZM,BM,PHIM = TT[conm],ZM[conm],BM[conm]/den_bright,PHIM[conm]
            TTp,ZP,BP,PHIP = TT[conp],ZP[conp],BP[conp]/den_bright,PHIP[conp]
            if plotme=='b vs phi':
                both_b_vs_phi(phip=PHIP,bp=BP,phim=PHIM,bm=BM,color=color,mylabel=mylabel,phi=PHI)
            elif plotme=='b vs z':
                both_b_vs_z(zp=ZP,bp=BP,zm=ZM,bm=BM,color=color,mylabel=mylabel,zc=zc_act)
            elif plotme=='b vs t':
                both_b_vs_t(tp=TTp,bp=BP,tm=TTm,bm=BM,color=color,mylabel=mylabel)
            elif plotme=='phi vs t':
                both_phi_vs_t(tp=TTp,phip=PHIP,tm=TTm,phim=PHIM,color=color,mylabel=mylabel,phi=PHI)
            elif plotme=='z vs t':
                both_z_vs_t(tp=TTp,zp=ZP,tm=TTm,zm=ZM,color=color,mylabel=mylabel,zc=zc_act)
#plt.legend(loc='upper right', prop={'size': 14})
#plt.xlabel('time since muon entry (ns)')
#plt.xticks(arange(0,40,1))
plt.xlim(18.3,36)
plt.show()

In [ ]:
len(T)

In [ ]:
TTm, PHIM

In [ ]:
plt.plot(TTm,ZM)

In [ ]:
print(1,2,3,4,5,sep="\t")

In [ ]:
#use it for phi vs t
plt.xticks([18,19,20,21,22,23,24,25,26,27])
plt.xlim(18.37,25.29

In [ ]:
PHIM

In [ ]:
ZM

In [ ]:
ZP

In [ ]:
PHIP

In [ ]:
detector

In [ ]:
pluslen

In [ ]:
minuslen

In [ ]:
len (ZM[conm])

In [ ]:
plt.plot(PHIM)